In [ ]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from tqdm import tqdm
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

#ktis19 modules
from starter_code.visualize import visualize
from starter_code.utils import load_case

#own modules
    #architecture of network
from modules.NeuralNetwork import makeModel
    #simple matplotlib visualizer with slider
from modules.visualizeSlider import cube_show_slider
    #preprocessing for data 
    #REDUNDANT - USED ONLY IN DATA GENERATORS
from modules.preprocessing import preprocess_X, preprocess_y, scale
    #data flow generators
from modules.dataGenerators import trainGenerator, validationGenerator

#changing to half precision due to memory limitations
import keras.backend as K
# K.set_floatx('float16')
# K.set_epsilon(1e-4)

In [ ]:
model = makeModel(512, 512, 2)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='accuracy'),
    tf.keras.callbacks.TensorBoard(log_dir='logs'),
    tf.keras.callbacks.ModelCheckpoint('saved_models/model1.h5', verbose=1, save_best_only=True)
]

0-179 Train set, 180-209 val_set, 210-299 test

In [ ]:
case_numbers = np.delete(np.arange(0, 209, 1), [158, 159, 170, 202])
case_numbers_val = case_numbers[179:]
case_numbers_train = case_numbers[:179]

AMOUNT OF SLICES FOR TRAINDATA == 38650

In [ ]:
batch_size = 16
data_size = 38650
results = model.fit(
    trainGenerator(case_numbers_train[:3], batch_size),
    steps_per_epoch = data_size/batch_size,
    epochs = 32,
    callbacks = callbacks,
    verbose = 1,
    validation_data = validationGenerator(case_numbers_val, batch_size)
    )

In [ ]:
model.save('saved_models\model_big_4_test')

In [ ]:
predicted = model.predict(X)

def dice_score(y_pred, y_test):
    return np.sum(y_pred)*2.0 / (np.sum(y_pred) + np.sum(y_test))
    
dice_score(predicted[:,:,:,1], y[:,:,:,1])

In [ ]:
%matplotlib qt
cube_show_slider(cube=predicted[:,:,:,0], axis=0, cmap='gray')